# TP3 Grupo 12

In [ ]:
# Importando librerías  
import pandas as pd 
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from collections import Counter

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
%matplotlib inline

import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier


Función para evaluación del modelo con regresión logística, variando datos de entrada

In [ ]:
def logistic_model(X_train, X_test, y_train, y_test):
    '''Creacióm del modelo de Regresión Logistica variando datos de entrada'''
    
    lr = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cholesky")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    lr.fit(X_train_scaled, y_train)
    return lr

In [ ]:
def mostrar_resultados(y_test, y_pred):
    confusion_matrix(y_test, y_test_pred)
    #plt.figure(figsize=(12, 12))
    sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt='.0f');
    plt.title("Confusion matrix")
    plt.ylabel('Verdaderos')
    plt.xlabel('Predichos')
    plt.show()
    print(classification_report(y_test, y_test_pred))

## 1. Lectura de DataFrame 

El DataFrame usado para este TP consta de datos provenientes de la plataforma Kaggle, en el que se evalúan algunas variables relacionadas con evaluación de posible fraude en transacciones bancarias realizadas con tarjetas de crédito. Se trata de un Dataset desbalanceado, respecto a la proporción en la variable target (fraud). 

In [ ]:
data = pd.read_csv(r'card_transdata.csv', sep=',')
print(f'El dataframe de transacciones está compuesto por {data.shape[1]} columnas y {data.shape[0]} filas') 
print(data.sample(3))
data.shape

## 2. Descripción breve del DataFrame.


El DataFrame en estudio cuenta con 8 columnas y un total de 1000000 filas (no tiene valores nulos). 

Las columnas contenidas en el DataFrame son: 

2.1 distance_from_home. Indica la distancia desde casa respecto al lugar en donde ocurrió la transacción. Ejemplo: 2.813104. 

2.2 distance_from_last_transaction. Indica la distancia entre el punto de la transacción con respecto a la última transacción registrada. Ejemplo: 0.153290.

2.3 ratio_to_median_purchase_price. es la relación entre el monto de la transacción y el precio medio de compra del cliente. Ejemplo: 1.135134.

2.4 repeat_retailer. Columna con variables dummies para indicar con 1 si la transacción se realizó desde el mismo minorista y con 0 si no. Ejemplo: 1.0.

2.5 used_chip. Columna con variables dummies para indicar con 1 si en la transacción se utilizó el chip de la tarjeta de crédito y con 0 si no se utilizó. Ejemplo: 0.0.

2.6 used_pin_number. Columna con variables dummies que indica con 1 si en la transacción se usó el número de pin y con 0 en caso contrario. Ejemplo: 1.0.

2.7 online_order. Columna con variables dummies en la que se indica con 1 si la transacción corresponde a un pedido en línea y 0 si no fue así. Ejemplo: 1.0. 

2.8 fraud. Columna con variable target que indica si la transacción fue identificada como fraude (1) o no (0). Ejemplo: 1.0.

In [ ]:
# Resumen de nombre y tipo de dato por columna

print('\n')
print('-----------Tipo de datos por columna-----------')
print(data.dtypes)

## 3. Distribución de los datos.  

In [ ]:
# Para saber valores no nulos

msno.bar(data)

Dentro del DataFrame no se encuentran valores nulos, todas las variables a usar en los diferentes modelos (KNN, Naives Bayes y Regresión Logística) tienen los los datos completos. 

## * Variable TARGET a Determinar

Según el análisis realizado y la propuesta original en el dataset, la variable target a considerar, en la construcción de los diferentes modelos,  será la columna "fraud".

In [ ]:
# Considerando lo anterior, resulta necesario conocer la proporción de la variable target
count_classes = data.fraud.value_counts(sort=True)
count_classes

Para una población de 1000000 de transacciones, tan sólo 87403 son etiquetadas como casos de fraude. Representan aproximadamente el 9% de las muestras.

In [ ]:
count_classes.plot(kind = 'bar', rot=0)
plt.title("Frecuencia de numero de observaciones")
plt.xlabel("Class")
plt.ylabel("Numero de observaciones");

# Propuestas 

1. Trabajar con dataset total. (1MM de registros, proporción variable target 90-10)
2. Seleccionar menor cantidad de registros y buscando una proporción de la variable target 70-30, para tener el datasets menos desbalanceado. Según cálculos , se debería extraer una muestra de 291343 datos totales. 


A partir de la selección del dataset: 

1. Aplicar los siguientes modelos sobre el conjunto de datos: 

   1.a. KNN. (Carlos)

   1.b. Naives Bayes. (Santiago)
   
   1.c. Regresión Logística. (Moises) 

In [ ]:
# Dataset de sólo no fraude 
df_nf = df.loc[df['fraud'] == 0]
df_nf.fraud.value_counts()
df_nf2 = df_nf.sample(203940)
print(df_nf2.fraud.value_counts())

# Generar otro dataset con los registros de fraude 
df_f = df.loc[df['fraud'] == 1]
df_f.fraud.value_counts()


In [ ]:
datos = pd.merge(df_nf2, df_f, how = 'outer')
datos.fraud.value_counts()

display(datos.sample(10))



## 4. Aplicando modelo de Regresión Logística

4.1 Full Datos. 
    
    Como 1era aproximación para la generación del modelo, será usado todo el volumne de datos. Sabiendo que la variable a predecir se encuentra desbalanceada.  

In [ ]:
# Para considerar las variables predictoras

feature_cols = [x for x in data.columns if ((x != 'fraud'))]
feature_cols

In [ ]:
# División de los datos en train y test
# ==============================================================================
X = data[feature_cols]
y = data['fraud']


X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        stratify=y,
                                        train_size   = 0.8,
                                        random_state = 123,
                                     )  

In [ ]:
# Estandarización
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
# Entrenando al modelo 
# Instanciamos un objeto de esa clase
logistic_regression = LogisticRegression()
# Ajustamos esta instancia con los datos de entrenamiento
logistic_regression.fit(X_train_scaled, y_train)
print(logistic_regression.coef_)
print(logistic_regression.intercept_)


In [ ]:
# Hacemos predicciones con el modelo entrenado
y_train_pred = logistic_regression.predict(X_train_scaled)
y_test_pred = logistic_regression.predict(scaler.transform(X_test)) # Notar que debemos escalar los datos de testeo antes de realizar predicciones

In [ ]:
# Elaboramos la matriz de confusión
confusion_matrix(y_test, y_test_pred)
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt='.0f')
plt.ylabel('Verdaderos')
plt.xlabel('Predichos');

In [ ]:
logistic_regression.predict_proba(X_test)

In [ ]:
logistic_regression.predict_proba(X_test).shape

In [ ]:
# Calculemos el accuracy
accuracy_score(y_test, y_test_pred)

In [ ]:
# Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_test_pred)
print('Sensibilidad del modelo:')
print(sensibilidad)

In [ ]:
puntajef1 = f1_score(y_test, y_test_pred)
print('Puntaje F1 del modelo:')
print(puntajef1)

In [ ]:
roc_auc = roc_auc_score(y_test, y_test_pred)
print('Curva ROC - AUC del modelo:')
print(roc_auc)

In [ ]:
# Haciendo un resumen de los parámetros obtenidos usando todo el dataset 
print(classification_report(y_test, y_test_pred))

En la matriz de confusión se obtienen alrededor de 7000 fallos y 10000 aciertos dando un recall de 0.60 y es el valor que queremos mejorar. También es interesante notar que en la columna de f1-score obtenemos muy buenos resultados, a pesar de ello deben ser considerados con ciertas reservas, pues están reflejando una realidad parcial. 

Estas métricas serán consideradas como línea base para comparación y búsqueda de mejora. Un aspecto a evaluar será el tema del hecho que el dataset se encuentra desbalanceado. 

## 4.2. Buscando balancear las clases

### 4.2.1 Penalización de clases mayoritarias

Según literatura de LogisticRegression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

- class_weight: dict or ‘balanced’, default=None
Weights associated with classes in the form {class_label: weight}. If not given, all classes are supposed to have weight one.

The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)).

Note that these weights will be multiplied with sample_weight (passed through the fit method) if sample_weight is specified.

- solver{‘lbfgs’, ‘liblinear’, ‘newton-cg’, ‘newton-cholesky’, ‘sag’, ‘saga’}, default=’lbfgs’
Algorithm to use in the optimization problem. Default is ‘lbfgs’. To choose a solver, you might want to consider the following aspects:
‘newton-cholesky’ is a good choice for n_samples >> n_features, especially with one-hot encoded categorical features with rare categories. Note that it is limited to binary classification and the one-versus-rest reduction for multiclass classification.



In [ ]:
penalidad = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cholesky",class_weight="balanced")
penalidad.fit(X_train_scaled, y_train)

y_test_pred = penalidad.predict(scaler.transform(X_test))

In [ ]:
mostrar_resultados(y_test, y_test_pred)

### 4.2.1 Undersampling (us) sobre la clase predominante (No Fraude)

In [ ]:
# us trabaja parecido a KNN
us = NearMiss(n_neighbors=3, version=1)
# Para cambiar proporciones podemos usar version = 3

X_train_res, y_train_res = us.fit_resample(X_train, y_train)
 
print(f'Distribución antes de resampling {Counter(y_train)}')
print(f'Distribución antes de resampling {Counter(y_train_res)}')

In [ ]:
model = logistic_model(X_train_res, X_test, y_train_res, y_test)
y_test_pred = model.predict(scaler.transform(X_test))

mostrar_resultados(y_test, y_test_pred)

Identifica todos los fraudes porque la probabilidad en el set de entrenamiento la proporción es 50/50. Aumentan los falsos positivos, disminuye el f1 score.

### 4.2.2 Oversampling (os) de la clase minoritaria (Fraude)

Para este caso se usará RandomOverSampler para crear nuevas muestras "sintéticas" para la clase 'Fraude'. 
Al usar 'sampling_strategy' = 0.5, estamos creando muestras sintéticas hasta alcanzar un numero de muestras igual al 50% de la clase mayoritaria (No Fraude), es decir 365039 casos Fraude.

In [ ]:
os =  RandomOverSampler(sampling_strategy = 0.5, random_state=1234)
X_train_res, y_train_res = os.fit_resample(X_train, y_train)
 
print(f'Distribución antes de resampling {Counter(y_train)}')
print(f'Distribución después de resampling {Counter(y_train_res)}')

In [ ]:
model = logistic_model(X_train_res, X_test, y_train_res, y_test)
y_test_pred = model.predict(scaler.transform(X_test))

mostrar_resultados(y_test, y_test_pred)

Aumentan los falsos negativos, disminuyen los falsos positivos, aumenta la precisión y el F1 score. Se identifican menor cantidad de casos de Fraude que el caso anterior. 

### 4.2.3 Aplicando modelos conjuntos. (Oversampling (os) y Undersampling (us)) -- Smote-Tomek 

* SMOTE (Synthetic Minority Over-sampling Technique) para oversampling. Puntos vecinos cercanos y agrega puntos “en linea recta” entre ellos. 

* Tomek para undersampling. Elimina los de distinta clase que sean nearest neighbor y deja ver mejor el decisión boundary (la zona limítrofe de las clases).

In [ ]:
os_us = SMOTETomek(sampling_strategy=0.5, random_state=1234)
X_train_res, y_train_res = os_us.fit_resample(X_train, y_train)
 
print(f'Distribución antes de resampling {Counter(y_train)}')
print(f'Distribución antes de resampling {Counter(y_train_res)}')

In [ ]:
model = logistic_model(X_train_res, X_test, y_train_res, y_test)
y_test_pred = model.predict(scaler.transform(X_test))

mostrar_resultados(y_test, y_test_pred)

Aumentan los falsos negativos. 

## 5. Aplicando modelo Naiyes Bayes

In [ ]:
X = data.iloc[:, 0:-1]
y = data.iloc[:, -1]

In [ ]:
data.head()

In [ ]:
# VAMOS A HACER UN PAIR PLOT PARA VER SI LOS DATOS ESTÁN BIEN DIFERENCIADOS ENTRE CLASES.
# PARA MAYOR VELOCIDAD AL GRAFICAR, USAREMOS UN DF MUY PEQUEÑO.
# PARA ENTRENAR EL MODELO SE USARÁ UNO MÁS GRANDE
x_train, x_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.99,
                                                    random_state=0,
                                                    stratify=y)

In [ ]:
y_train.value_counts()

In [ ]:
# DE IGUAL FORMA, SE ESCALARÁN LOS DATOS PARA VISUALIZAR MEJOR EL PAIR PLOT.
ss = StandardScaler()
xscaled = pd.DataFrame(ss.fit_transform(x_train),columns = x_train.columns)

In [ ]:
# LAS CLASES PARECEN ESTAR BIEN SEPARADAS PARA ALGUNAS VARIABLES 
# POR LO QUE ES VIABLE HACER UN MODELO DE CLASIFICACIÓN.
pd.plotting.scatter_matrix(xscaled, c=y_train, figsize=(20,20),
                           marker='o', )

### 5.1. Naive Bayes (Todos los datos)

In [ ]:
# VOLVEMOS A SEPARAR EN TRAIN Y TEST PERO ESTA VEZ
x_train, x_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=123,
                                                    stratify=y)

In [ ]:
model1 = GaussianNB()
model1.fit(x_train, y_train); 

In [ ]:
model2 = MultinomialNB()
model2.fit(x_train, y_train)

In [ ]:
y_pred_gnb = model1.predict(x_test)
y_pred_mnb = model2.predict(x_test)

In [ ]:
print (f'Accuracy para modelo gaussiano: {accuracy_score(y_pred_gnb,y_test)}')
print (f'Accuracy para modelo multinomial: {accuracy_score(y_pred_mnb,y_test)}')
print()
print (f'Recall para modelo gaussiano: {recall_score(y_test,y_pred_gnb)}')
print (f'Recall para modelo multinomial: {recall_score(y_test,y_pred_mnb)}')

In [ ]:
g = sns.heatmap(confusion_matrix(y_test,model1.predict(x_test)),annot=True, fmt='.0f')
g.set_xlabel('predicho')
g.set_ylabel('real')
g.set_title('GaussianNB')

In [ ]:
g = sns.heatmap(confusion_matrix(y_test,model2.predict(x_test)),annot=True, fmt='.2f')
g.set_xlabel('predicho')
g.set_ylabel('real')
g.set_title('MultinomialNB')

### 5.2. Naive Bayes (Balanceado)

### Para este modelo sólo será construido un DataFrame considerando la siguiente premisa: 
* Seleccionar menor cantidad de registros y buscando una proporción de la variable target 70-30, para tener el datasets menos desbalanceado. Según cálculos , se debería extraer una muestra de 291343 datos totales. 

In [ ]:
# Dataset de sólo no fraude 
df_nf = data.loc[df['fraud'] == 0]
df_nf.fraud.value_counts()
df_nf2 = df_nf.sample(203940, random_state=123)
print(df_nf2.fraud.value_counts())

# Generar otro dataset con los registros de fraude 
df_f = data.loc[df['fraud'] == 1]
df_f.fraud.value_counts()

In [ ]:
definitivo = pd.merge(df_nf2, df_f, how = 'outer')
definitivo.fraud.value_counts()

display(definitivo.sample(10))

In [ ]:
X = definitivo.iloc[:, 0:-1]
y = definitivo.iloc[:, -1]


x_train, x_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=123,
                                                    stratify=y)

In [ ]:
model1 = GaussianNB()
model1.fit(x_train, y_train);  

In [ ]:
model2 = MultinomialNB()
model2.fit(x_train, y_train)

In [ ]:
y_pred_gnb = model1.predict(x_test)
y_pred_mnb = model2.predict(x_test)

In [ ]:
print (f'Accuracy para modelo gaussiano: {accuracy_score(y_pred_gnb,y_test)}')
print (f'Accuracy para modelo multinomial: {accuracy_score(y_pred_mnb,y_test),}')
print()
print (f'Recall para modelo gaussiano: {recall_score(y_test,y_pred_gnb)}')
print (f'Recall para modelo multinomial: {recall_score(y_test,y_pred_mnb)}')

In [ ]:
g = sns.heatmap(confusion_matrix(y_test,model1.predict(x_test)),annot=True, fmt='.2f')
g.set_xlabel('predicho')
g.set_ylabel('real')
g.set_title('GaussianNB')

In [ ]:
g = sns.heatmap(confusion_matrix(y_test,model2.predict(x_test)),annot=True, fmt='.2f')
g.set_xlabel('predicho')
g.set_ylabel('real')
g.set_title('MultinomialNB')

## 6. Aplicando modelo KNN

### 6.1. KNN (Todos los datos)

In [ ]:
# Construimos la matriz de features donde tomamos todo el DataFrame en la eje X
X = data.iloc[:, 0:-1]
# Construimos el vector target Y con la columna Fraud del DF
y = data['fraud']

In [ ]:
## Serán generados los datos de entrenamiento y test
# El argumento stratify nos permite generar una división que respeta la misma proporción entre clases en ambos sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=12)

In [ ]:
# Clasificando el dataframe con el modelo KNN 
# Inicialmente tomamos el hiperparametro por defecto, sin estandarizar la matriz

knn = KNeighborsClassifier()
# Ajustamos a los datos de entrenamiento
knn.fit(X_train, y_train)

# Predecimos etiquetas para los datos de test
y_pred = knn.predict(X_test)

#### Entre los datos de pruebas y prediccion, vemos que se obtiene un 98% de Precision de conincidencia entre la muestra real y la prediccion

In [ ]:
# Evaluamos el accuracy del modelo
accuracy_score(y_test, y_pred).round(2)

#### Optimizamos el valor del hiperparametro K, haciendo unsando recall_score tomando un rango de 1 a 10 para posterioirmente determinar el valor optimo en K

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
#optimizar el valor de K

scores_para_df = []

for i in tqdm(range(1,10)):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    dict_row_score = {'recall': recall_score(y_test, y_pred), 'n_neighbors':i}
    # Guardamos cada uno en la lista de diccionarios
    scores_para_df.append(dict_row_score)

In [ ]:
df_scores = pd.DataFrame(scores_para_df)
df_scores.head()

In [ ]:
# Graficamos los resultados
plt.plot(df_scores['n_neighbors'], df_scores['recall'], color='g');
plt.xlabel('n_neighbors'); plt.ylabel('Recall')
plt.grid(); plt.show()

#### Validamos cual es el valor que maximiza el score de recall, siendo en este caso el n_neighbors= 1

### 6.2. Estandarizacion de los datos de train

In [ ]:
#Estandarizacion Matriz
# Utilizamos sklearn para estandarizar la matriz de features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
# Verificamos que las variables ahora tengan media 0 y desvío 1.
print('Medias:', np.mean(X_train, axis=0).round(2))
print('Desvio:', np.std(X_train, axis=0).round(2))

In [ ]:
# Calculamos nuevamente los scores de Recall pero esta vez sobre los features estandarizados:

scores_para_df_standard = []

for i in tqdm(range(1,10)):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    dict_row_score = {'recall': recall_score(y_test, y_pred), 'n_neighbors':i}
    # Guardamos cada uno en la lista de diccionarios
    scores_para_df_standard.append(dict_row_score)

In [ ]:
# Creamos el DataFrame a partir de la lista de diccionarios
df_scores_standard = pd.DataFrame(scores_para_df_standard)
df_scores_standard

In [ ]:
# Graficamos los resultados
plt.plot(df_scores_standard['n_neighbors'], df_scores_standard['recall'], color='g');
plt.xlabel('n_neighbors'); plt.ylabel('Recall')
plt.grid(); plt.show()

Ahora obteniendo el valor optimo, conseguido en el paso anterioir, hacemos de nuevo el entramiento de los datos

In [ ]:
# Asignamos el valor del k óptimo a una variable
best_k = df_scores_standard.loc[df_scores.recall == df_scores.recall.max(), 'n_neighbors'].values[0]
best_k

In [ ]:
# Elegimos el modelo óptimo de acuerdo a las pruebas de recall_score
model = KNeighborsClassifier(n_neighbors=best_k)

# Lo ajustamos sobre los datos de entrenamiento
model.fit(X_train, y_train)

Obtenemos un ajuste cerca de 100%

In [ ]:
# Evaluamos qué accuracy obtenemos en train
accuracy_score(y_train, model.predict(X_train)).round(2)

In [ ]:
# Generación de la matriz de confusión
cm = confusion_matrix(y_test, y_pred)
cm

sns.heatmap(cm, annot=True, fmt='.0f')
plt.title("Confusion matrix")
plt.ylabel('Real')
plt.xlabel('Predicción');
plt.show()

* Casos negativos reales 62.397, correspondientes a la primera fila.

* Casos positivos reales 14.318, correspondientes a la segunda fila.

* Casos negativos predichos, corresponden a la primera columna.

* Casos positivos predichos, corresponden a la segunda columna.

* Diagonal en la matriz corresponde a las clasificaciones correctas. Siendo 62.397 casos negativos y 14.318 a casos positivos de Fraude.
Los Elementos de la diagonal inversa, nuestro modelo los confude siendo 165.752 casos mal etiquetado para negativo y 7.533 para positivo de fraude.

* Finalmente calculamos la exactitud de nuestro modelo, de manera manual y computada, siendo ambas de precision del 92%

In [ ]:
((cm[0,0] + cm[1,1]) / len(y_pred)).round(2)

In [ ]:
accuracy_score(y_test, y_pred).round(2)

### 6.3 Balanceo de clases

Partiendo desde la misma premisa considerada para Naives Bayes, será usado el DataFrame generado de manera manual


In [ ]:
# Preparar matriz de *features* y vector *target*
X = definitivo.iloc[: , :7]
y = definitivo['fraud']

In [ ]:
# Elegir los hiperparámetros del modelo. Instanciamos el modelo con su configuración por defecto
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
# Separar los sets de entrenamiento y de testing
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=2)

# Ajustamos a los datos de entrenamiento.
knn.fit(X_train, y_train);

# Predecimos etiquetas para los datos de test.
y_pred = knn.predict(X_test)

In [ ]:
# # Evaluamos el accuracy del modelo
accuracy_score(y_test, y_pred).round(2)

 Nuestro modelo puede clasificar correctamente el 97% de las observaciones del dataset de test.

In [ ]:
# Forma 1.
# # Ahora, vamos a evaluar el modelo

X_train_train, X_validation, y_train_train, y_validation = train_test_split(X_train, y_train, random_state=2)

k_range = (range(1, 25))
scores = []
for k in tqdm(k_range):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_train, y_train_train)
    y_pred = knn.predict(X_validation)
    scores.append(accuracy_score(y_validation, y_pred))  

In [ ]:
# Graficando la mejor busqueda del hiperparámetro

plt.figure(figsize=(4,3))
plt.plot(k_range, scores)
plt.xlabel('Valor de K'); plt.ylabel('Test Accuracy')
plt.grid(); plt.show()

In [ ]:
# Forma 2.
# Ahora, vamos a evaluar el modelo

from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=5, shuffle=True, random_state=12)

scores_para_definitivo = []

for i in range(1, 21):
    
    model = KNeighborsClassifier(n_neighbors=i)
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=kf)

    dict_row_score = {'score_medio':np.mean(cv_scores),
                      'score_std':np.std(cv_scores), 'n_neighbors':i}
    
    scores_para_definitivo.append(dict_row_score)

In [ ]:
# Creamos el DataFrame a partir de la lista de diccionarios
df_scores = pd.DataFrame(scores_para_definitivo)

In [ ]:
# Generamos los límites inferior y superior
df_scores['limite_inferior'] = df_scores['score_medio'] - df_scores['score_std']
df_scores['limite_superior'] = df_scores['score_medio'] + df_scores['score_std']

In [ ]:
# Graficamos los resultados
plt.plot(df_scores['n_neighbors'], df_scores['limite_inferior'], color='r')
plt.plot(df_scores['n_neighbors'], df_scores['score_medio'], color='b')
plt.plot(df_scores['n_neighbors'], df_scores['limite_superior'], color='r');

In [ ]:
# Identificamos el score máximo
df_scores.loc[df_scores.score_medio == df_scores.score_medio.max()]